###### <img src="Electronic_Brain.png" width="200" style="float:left">
<h1> Spring 2021 ML Course.</h1>
<h2> Exercise 13: Periodicity Detection via Clustering<br>Tools: HDBScan</h2>

In [ ]:
import numpy as np
import pandas as pd

%matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from sklearn import preprocessing
from itertools import product
import hdbscan

<img src="desktop-computer-icon.png" width="90" style="float:left; margin-right: 10px;">
<h1> &nbsp; Section A: Signal Generation.</h1>
We assume an unknown source has supplied us with candidate detections in the frequency spectrum, providing us (only) with a bounding rectangle.<br>
Our mission is to decide how many emitters are responsible for the detections, and to provide [detection / emitter] pairing.<br>
We now generate and take a first look at our data.

In [ ]:
np.random.seed(6)
snapshot_dims = {'x':1000, 'y':1000}  # The dimensions of our spectrogram.
emitters = [
    {
        'name' : 'emitter_1_high_bursts',
        'type' : 'const_freq',
        'f_start'     : 500,
        'f_bw'        : 50,
        'fs_sigma'    : 2,
        'f_bw_sigma'  : 2,
        'num_reps'    : 8,
        't_start'     : 50,
        't_duration'  : 20,
        't_jump'      : 75,
        'ts_sigma'    : 1,
        't_dur_sigma' : 1,
        'color'       : 'red',
    },
    {
        'name' : 'emitter_1_low_bursts',
        'type' : 'const_freq',
        'f_start'     : 250,
        'f_bw'        : 50,
        'fs_sigma'    : 2,
        'f_bw_sigma'  : 2,
        'num_reps'    : 8,
        't_start'     : 70,
        't_duration'  : 20,
        't_jump'      : 75,
        'ts_sigma'    : 1,
        't_dur_sigma' : 1,
        'color'       : 'red',
    },
    {
        'name' : 'hopper_1',
        'type' : 'rand_freq',
        'f_min'       : 100,
        'f_max'       : 700,
        'f_bw'        : 200,
        'fs_sigma'    : 2,
        'f_bw_sigma'  : 2,
        'num_reps'    : 12,
        't_start'     : 50,
        't_duration'  : 50,
        't_jump'      : 60,
        'ts_sigma'    : 0.1,
        't_dur_sigma' : 0.1,
        'color'       : 'blue',
    },
    {
        'name' : 'emitter_2',
        'type' : 'const_freq',
        'f_start'    : 100,
        'f_bw'       : 75,
        'fs_sigma'   : 1,
        'f_bw_sigma' : 3,
        'num_reps'   : 9,
        't_start'    : 20,
        't_duration' : 30,
        't_jump'     : 100,
        'ts_sigma'   : 1,
        't_dur_sigma': 1,
        'color'      : 'darkgoldenrod',
    },
        {
        'name' : 'emitter_3',
        'type' : 'const_freq',
        'f_start'     : 750,
        'f_bw'        : 75,
        'fs_sigma'    : 1,
        'f_bw_sigma'  : 3,
        'num_reps'    : 6,
        't_start'     : 20,
        't_duration'  : 20,
        't_jump'      : 140,
        'ts_sigma'    : 1,
        't_dur_sigma' : 1,
        'color'       : 'green',
    },
    # {
    #     'name' : 'noise_1',
    #     'type' : 'const_freq',
    #     'f_start'     : 450,
    #     'f_bw'        : 50,
    #     'fs_sigma'    : 1,
    #     'f_bw_sigma'  : 1,
    #     'num_reps'    : 1,
    #     't_start'     : 620,
    #     't_duration'  : 60,
    #     't_jump'      : 140,
    #     'ts_sigma'    : 1,
    #     't_dur_sigma' : 1,
    #     'color'       : 'aqua',
    # },
    #     {
    #     'name' : 'noise_2',
    #     'type' : 'const_freq',
    #     'f_start'     : 650,
    #     'f_bw'        : 50,
    #     'fs_sigma'    : 1,
    #     'f_bw_sigma'  : 1,
    #     'num_reps'    : 1,
    #     't_start'     : 320,
    #     't_duration'  : 100,
    #     't_jump'      : 140,
    #     'ts_sigma'    : 1,
    #     't_dur_sigma' : 1,
    #     'color'       : 'magenta',
    # },
]

In [ ]:
# Given an emitter, generate "objects": rectangles with start / end times and frequencies (names and colors as well).
def generate_objects(obj_params):
    out_objects = []
    for i in range(obj_params['num_reps']):
        
        # Object starting times are period, with t_jump between each two, with a bit of noise.
        t_start = obj_params['t_start'] + i*obj_params['t_jump'] + int(np.random.normal(0, obj_params['ts_sigma']))
        
        if obj_params['type']=='const_freq':
            # Constant frequency: only add a bit of "frequency noise".
            f_start = obj_params['f_start'] + int(np.random.normal(0, obj_params['fs_sigma']))            
        else:
            # Hopping frequency emitter: randomly generate a frequency inside the given range.
            f_start = int(np.random.randint(obj_params['f_min'], obj_params['f_max']))

        curr_obj = {
            't_start' : t_start + int(np.random.normal(0, obj_params['ts_sigma'])),
            't_end'   : t_start + obj_params['t_duration'] + int(np.random.normal(0, obj_params['t_dur_sigma'])),
            'f_start' : f_start + int(np.random.normal(0, obj_params['fs_sigma'])),
            'f_end'   : f_start + obj_params['f_bw'] + int(np.random.normal(0, obj_params['f_bw_sigma'])),
            'name'    : obj_params['name'],
            'type'    : obj_params['type'],
            'color'   : obj_params['color'],
        }
        out_objects.append(curr_obj)
    return(out_objects)

In [ ]:
# Construct the  objects, turn them into a nice DataFrame.
all_objects = []
for emitter in emitters:
    all_objects.extend(generate_objects(emitter))

# Form the dataframe, dropping a random fraction of objects.
obj_frac_to_drop = 0
obj_df = pd.DataFrame.from_dict(all_objects)
obj_df.drop(obj_df.sample(frac=obj_frac_to_drop).index, inplace=True)

# Add a few "natural" features (bandwidth, central frequency, duration).
obj_df['f_bw'] = obj_df['f_end'] - obj_df['f_start']
obj_df['f_cent'] = 0.5 * (obj_df['f_end'] + obj_df['f_start'])
obj_df['t_duration'] = obj_df['t_end'] - obj_df['t_start']

In [ ]:
# Show the raw detections.
fig = plt.figure()
fig.suptitle('Periodic transmissions from 4 emitters, highlighted by different colors\n' + \
            '(obviously, the chaining algorithm doesn\'t \"know\" the colors)\n' + \
            'Missing detections: ' + str(round(100*obj_frac_to_drop, 2)) + '%')
for _, obj in obj_df.iterrows():
    rect_t = obj['t_start'] / snapshot_dims['x']
    rect_f = obj['f_start'] / snapshot_dims['y']
    obj_width = obj['t_duration'] / snapshot_dims['x']
    obj_f_bw = obj['f_end'] - obj['f_start']
    obj_height = obj_f_bw / snapshot_dims['y']
    p = patches.Rectangle((rect_t, rect_f), obj_width, obj_height, fill=True, color=obj['color'], alpha=0.4)
    fig.add_artist(p)
fig.show()

<img src="desktop-computer-icon.png" width="90" style="float:left; margin-right: 10px;">
<h1> &nbsp; Section B: Preprocessing.</h1>
Scale (i.e., normalize) the data to zero mean / unit variance.<br>
Obviously, this step is crucial before applying any geometry-based algorithm.

In [ ]:
bw_scaler = preprocessing.StandardScaler().fit(obj_df['f_bw'].values[:, None])
f_cent_scaler = preprocessing.StandardScaler().fit(obj_df['f_cent'].values[:, None])
t_dur_scaler = preprocessing.StandardScaler().fit(obj_df['t_duration'].values[:, None])

f_bw_norm = bw_scaler.transform(np.array(obj_df['f_bw'])[:, None])
f_cent_norm = f_cent_scaler.transform(np.array(obj_df['f_cent'])[:, None])
t_duration_norm = t_dur_scaler.transform(np.array(obj_df['t_duration'])[:, None])

# Place the resulting scaled versions of the vars in a new dataframe.
norm_objs = [
    {'f_bw_norm' : f_bw_norm[i],
     'f_cent_norm' : f_cent_norm[i],
     'f_duration_norm' : t_duration_norm[i]} for i in range(len(f_bw_norm))]
norm_objs_df = pd.DataFrame.from_dict(norm_objs)

<img src="cluster-analysis.png" width="90" style="float:left; margin-right: 10px;">
<h1> &nbsp; Section C: Clustering.</h1>

In [ ]:
# Cluster the 3D dataframe using the HDBscan* alg.
# Exercise: try out various other metrics (for example 'cityblock').
# A list of different metrics can be found in the docs:
# https://hdbscan.readthedocs.io/en/latest/basic_hdbscan.html#what-about-different-metrics
clusterer = hdbscan.HDBSCAN()
clusterer.fit(norm_objs_df)
print("Cluster output:\n", clusterer.labels_)
print("Probabilities:\n", clusterer.probabilities_)

# Assign the cluster labels back to the original dataframe (preserves order).
obj_df['cluster_label'] = clusterer.labels_

In [ ]:
# Show the clustering results in 3D.
fig = plt.figure()
fig.suptitle('Objects detected (colors denote clusters)')
ax = fig.add_subplot(projection='3d')
ax.scatter(obj_df['t_duration'], obj_df['f_cent'], obj_df['f_bw'], c=obj_df['cluster_label'])
ax.set_xlabel('Object Duration')
ax.set_ylabel('Central Frequency')
ax.set_zlabel('Frequency BW');

<img src="desktop-computer-icon.png" width="90" style="float:left; margin-right: 10px;">
<h1> &nbsp; Exercise</h1>
Please play around with the parameters and see when the algorithm starts "breaking":<br>
- Noise levels, as expressed by the various "*_sigma" parameters.<br>
- Percentage of objects dropped, as expressed by the obj_frac_to_drop parameter.<br>
- Noise detections, such as the ones commented out in the first cell of Section A.

<img src="Time.png" width="90" style="float:left; margin-right: 10px;">
<h1> &nbsp; Section D: Extracting time-to-next-object (TTNO) statistics.</h1>

In [ ]:
"""
Extract time-to-next-object (TTNO) from each object to each cluster.
Notice the TTNO statistic is also defined from an object to its own cluster,
which can (and should) be used to test cluster validity.
Periodic clusters (from by a single periodic emitter) will manifest as peaky and unimodal densities.
Other clusters (from two or more emitters) will manifest as spread and multimodal.
We can (and should) use statistical tests to merge and / or split clusters downstream
[if interested, look up entropy and the "dip statistic" in this week's YouTube links].
"""
cluster_labels = pd.unique(obj_df['cluster_label'])  # Extract the set of unique cluster labels.

# Remove the noise cluster (labeled as -1), i.e., points the algorithm hasn't assigned to any cluster (we will not be using them).
cluster_labels = np.delete(cluster_labels, np.where(cluster_labels==-1))

# At this point we have already clustered the points.
# We now go back to using the original objects dataframe.
for i, obj in obj_df.iterrows():
    for clust_label in cluster_labels:
        
        # Define the "future objects" dataframe, which for each object and each cluster,
        # selects objects (from the original dataframe) with the required cluster label, starting after the object.
        future_objs_df = obj_df.loc[(obj_df['t_start'] > obj['t_start']) & (obj_df['cluster_label'] == clust_label)]

        # If no future objects are avail, move on.
        if future_objs_df.empty:
            continue

        # Otherwise, extract the TTNO.
        ttno = future_objs_df['t_start'].min() - obj['t_start']
        ttno_label = 'ttno_' + str(clust_label)
        obj_df.at[i, ttno_label] = ttno

In [ ]:
"""
Show the TTNO histograms for each pair of clusters.
First, we need to extract the raw stats.
"""
ttno_lists = {}
for clust_pair in product(cluster_labels, repeat=2):
    ttno_clust_label = 'ttno_' + str(clust_pair[1])
    ttno_list = obj_df.loc[(obj_df['cluster_label'] == clust_pair[0]) & (obj_df[ttno_clust_label].notna()), ttno_clust_label]

    if clust_pair[0] not in ttno_lists:
        ttno_lists[clust_pair[0]] = {}
    if clust_pair[1] not in ttno_lists[clust_pair[0]]:
        ttno_lists[clust_pair[0]][clust_pair[1]] = []

    ttno_lists[clust_pair[0]][clust_pair[1]] = ttno_list.values.tolist()
    # print("Cluster", clust_pair[0], "to", clust_pair[1], "TTNOs:", ttno_list.values.tolist())

# Show the histograms / KDE plots:
fig, axs = plt.subplots(nrows=len(cluster_labels),
                        ncols=len(cluster_labels),
                        subplot_kw={'xticks':[], 'yticks':[]})

for i, ax in enumerate(axs.flat):
    ax_col = i % len(cluster_labels)  # Modulo operation "finds" the correct column.
    ax_row = i // len(cluster_labels) # Integer division "finds" the correct row.

    # The addition of the normally distributed component avoids the pesky "0 variance" problem
    # histogram_data = np.array(ttno_lists[ax_row][ax_col]) + np.random.normal(0, 0.01, len(ttno_lists[ax_row][ax_col]))
    ax.hist(ttno_lists[ax_row][ax_col], bins=range(150), density=True)  # A standard histogram.

fig.suptitle('Time-to-Next-Object (TTNO) Distributions for Cluster Pairs\n' + \
            'Clusters from a single periodic emitter manifest in peaky and unimodal TTNO density distributions\n' + \
            'Other clusters\' density distributions (from two or more emitters) appear spread and multimodal\n' + \
            '(NOTE the various KDE distributions are *NOT* equally scaled!)', fontsize=8);